In [1]:
from tqdm import tqdm
from sklearn.decomposition import NMF
import pandas as pd
from scipy import sparse
import numpy as np
import implicit

# read data
ratings = pd.read_feather('data/amazon_review_ratings.feather')

# indexing ids
# userid
userid_unique = pd.Series(ratings["userId"].unique())
index_userid_dict = userid_unique.to_dict()
# inverse
userid_index_dict = dict(map(reversed, index_userid_dict.items()))

# itemid
itemid_unique = pd.Series(ratings["itemId"].unique())
index_itemid_dict = itemid_unique.to_dict()
# inverse
itemid_index_dict = dict(map(reversed, index_itemid_dict.items()))

ratings["userId_reindex"] = ratings["userId"].map(userid_index_dict)
ratings["itemid_reindex"] = ratings["itemId"].map(itemid_index_dict)

# reindexしたidを使って、アイテムとジャンルの対応が取れるdictを作る
itemid_genres_dict = ratings[['itemid_reindex', 'category']].set_index('itemid_reindex')['category'].to_dict()

In [2]:
import cloudpickle
X_train = cloudpickle.load(open("output/Amazon-X_train.pkl","rb"))
X_test = cloudpickle.load(open("output/Amazon-X_test.pkl","rb"))
test_movies_and_TVs_pos_items_dict = cloudpickle.load(open('output/test_movies_and_TVs_pos_items_dict.pkl', 'rb'))

In [3]:
# aggregateのtrainをbookとmoviesに分離する
# bookの列
book_columns = [v for v in range(X_train.shape[1]) if 'book' in itemid_genres_dict[v]]
# moviesの列
movies_columns = [v for v in range(X_train.shape[1]) if 'movies_and_TVs' in itemid_genres_dict[v]]

# 選んだカラムに応じてとってくる
book_train = X_train[:, book_columns]
movies_train = X_train[:, movies_columns]

# moviesのみ、アイテムidのconcatとの対応関係が必要なので辞書として持っておく
movies_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'movies_and_TVs' in itemid_genres_dict[v]:
        movies_concat_itemid_dict[v] = count
        count += 1

In [4]:
# アイテムidのconcatとの対応関係が必要なので辞書として持っておく
book_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'book' in itemid_genres_dict[v]:
        book_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_book_concat_itemid_dict = dict(map(reversed, book_concat_itemid_dict.items()))

movies_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'movies_and_TVs' in itemid_genres_dict[v]:
        movies_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_movies_concat_itemid_dict = dict(map(reversed, movies_concat_itemid_dict.items()))

In [5]:
# それぞれにアクションしていないユーザを削る
# 全ユーザと、削ったあとでの対応関係を辞書として持っておく
book_train_selected = book_train[book_train.getnnz(1)>0]
movies_train_selected = movies_train[movies_train.getnnz(1)>0]

book_train_action_users = {}
book_users = book_train.getnnz(1)>0
count = 0
for i in range(book_train.shape[0]):
    if book_users[i]:
        book_train_action_users[i] = count
        count += 1

# inverse
inverse_book_train_action_users = dict(map(reversed, book_train_action_users.items()))

movies_train_action_users = {}
movies_users = movies_train.getnnz(1)>0
count = 0
for i in range(movies_train.shape[0]):
    if movies_users[i]:
        movies_train_action_users[i] = count
        count += 1

# inverse
inverse_movies_train_action_users = dict(map(reversed, movies_train_action_users.items()))

In [6]:
# bookだけでALSをまず行う
# mediateでの結果をそのまま持ってくる
import cloudpickle
book_ALS = cloudpickle.load(open('output/book_ALS.pkl', 'rb'))

In [7]:
# side informationとしてaction_NMFで得られたベクトルを使う
user_attributes = pd.DataFrame(book_ALS.user_factors)
user_attributes['UserId'] = user_attributes.index
# useridをconcatの次元に戻す
user_attributes['UserId'] = user_attributes['UserId'].map(inverse_book_train_action_users)

item_attributes = pd.DataFrame(book_ALS.item_factors)
item_attributes['ItemId'] = item_attributes.index
# itemidをconcatの次元に戻す
item_attributes['ItemId'] = item_attributes['ItemId'].map(inverse_book_concat_itemid_dict)

In [8]:
# X_trainから縦持ちに復元する(X_trainが既に横持ちなので)
userid_stacked = []
itemid_stacked = []
ratings_stacked = []

for i in tqdm(range(X_train.shape[0])):
    ratings = X_train[i, :].data
    item_idx = X_train[i, :].indices
    for idx, v in enumerate(item_idx):
        if 'movies' in itemid_genres_dict[v]:
            userid_stacked.append(i)
            itemid_stacked.append(v)
            ratings_stacked.append(ratings[idx])

movies_ratings_stacked = pd.DataFrame()
movies_ratings_stacked['UserId'] = userid_stacked
movies_ratings_stacked['ItemId'] = itemid_stacked
movies_ratings_stacked['Rating'] = ratings_stacked

100%|██████████| 690240/690240 [03:36<00:00, 3187.44it/s]


In [9]:
from cmfrec import CMF

# fitting a model and making some recommendations
recommender = CMF(k=20, k_main=3, k_user=2, k_item=1, reg_param=1e-4)
recommender.fit(ratings=movies_ratings_stacked, user_info=user_attributes, item_info=item_attributes,
                cols_bin_user=None, cols_bin_item=None)

/home/fufufukakaka/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fufufukakaka/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fufufukakaka/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 1.169096
  Number of iterations: 145
  Number of functions evaluations: 159


In [10]:
from lib.recommend_util import ndcg
ndcgs = {
    'ndcg5':  [],
    'ndcg10':  [],
    'ndcg20':  [],
    'ndcg50':  [],
    'ndcg100':  []
}
count = 0
for userid, pos_items in tqdm(test_movies_and_TVs_pos_items_dict.items()):
    pos_items = np.array(pos_items)
    try:
        recommended_items = np.array(recommender.topN(user=userid, n=100))
        ndcgs['ndcg5'].append(ndcg(recommended_items[:5], pos_items))
        ndcgs['ndcg10'].append(ndcg(recommended_items[:10], pos_items))
        ndcgs['ndcg20'].append(ndcg(recommended_items[:20], pos_items))
        ndcgs['ndcg50'].append(ndcg(recommended_items[:50], pos_items))
        ndcgs['ndcg100'].append(ndcg(recommended_items[:100], pos_items))
    except:
        count += 1
        # 推薦できないユーザの場合は無条件で0を入れる
        ndcgs['ndcg5'].append(0)
        ndcgs['ndcg10'].append(0)
        ndcgs['ndcg20'].append(0)
        ndcgs['ndcg50'].append(0)
        ndcgs['ndcg100'].append(0)

100%|██████████| 10339/10339 [00:57<00:00, 181.31it/s]


In [11]:
print("ndcg@5: {}".format(np.mean(ndcgs['ndcg5'])))
print("ndcg@10: {}".format(np.mean(ndcgs['ndcg10'])))
print("ndcg@20: {}".format(np.mean(ndcgs['ndcg20'])))
print("ndcg@50: {}".format(np.mean(ndcgs['ndcg50'])))
print("ndcg@100: {}".format(np.mean(ndcgs['ndcg100'])))

ndcg@5: 0.004390742224187599
ndcg@10: 0.007177840883149365
ndcg@20: 0.008616386034564395
ndcg@50: 0.013092420348606262
ndcg@100: 0.017197446044378967
